In [1]:
import requests
import json
from datetime import datetime
import time
import geocoder
import geopy
import math
from geopy.geocoders import Nominatim
import haversine 
from haversine import Unit
from dotenv import load_dotenv
import os


In [2]:
class Where_am_I():

    def __init__(self, nome=None):
        
        self.nome = nome
        self.current_time = datetime.now().strftime('%H:%M:%S')

    def current_location_name (self):

        return geocoder.ip('me')
    
    def my_coordinates(self):

        self.latitude = geocoder.ip('me').lat
        self.longitude = geocoder.ip('me').lng

        return self.latitude , self.longitude
    

In [3]:
class Iss():

    def __init__ (self):

        self.response = requests.get("http://api.open-notify.org/iss-now.json").json() 
        self.iss_time = datetime.fromtimestamp(self.response['timestamp']).strftime('%H:%M:%S')


    def whos_aboard (self):

        if self.resonse["message"] == "success": 

            return [names['name'] for names in requests.get("http://api.open-notify.org/astros.json").json()['people'] if names['craft'] == 'ISS']
        
        else:

            return "Out of orbit."

    def position_iss (self):

        self.iss_latitude = float(self.response['iss_position']['latitude'])
        
        self.iss_longitude = float(self.response['iss_position']['longitude'])

        self.lat_iss = "North" if self.iss_latitude > 0 else "South"

        self.long_iss =  "East" if self.iss_longitude > 0 else "West"


        return self.lat_iss, self.iss_latitude , self.long_iss, self.iss_longitude 
    
    def iss_complete_location(self):
        
        try:

            self.geolocation = Nominatim(user_agent="geoapiExercises")

            self.location = self.geolocation.reverse(str(self.position()[0])+","+str(self.position()[1])).raw['address']

            return self.location     

        except:
            
            return "Did the ISS enter a black hole?"
        
    def iss_country(self):

        self.contry_loc = None

        self.country_loc = self.iss_complete_location()['country'] if 'country' in self.iss_complete_location() else "--"

        return self.country_loc

In [4]:
class Astrolabe(Iss, Where_am_I):
    
    def __init__(self):

        super().__init__()


    def distance_km(self):   

       return round((haversine.haversine(self.position_iss(),self.my_coordinates(), unit = Unit.KILOMETERS)),2)
    

In [7]:
ISS_Monitoring().iss_database_connection()

'Success!'

In [6]:
import psycopg2


class ISS_Monitoring(Astrolabe, Iss):

    def __init__(self):

        super().__init__()

        self.user = os.environ.get("PSQL_USER")
        self.password = os.environ.get("PSQL_PASSWORD")
        self.host = os.environ.get("PSQL_HOST")
        self.port = os.environ.get("PSQL_PORT")
        self.database = os.environ.get("PSQL_DATABASE")
        
        self.connection = None


    def iss_database_connection(self):

        try:

            self.connection = psycopg2.connect(
                user = self.user,
                password = self.password,
                host = self.host,
                port = self.port,
                database = self.database
            )

            return "Success!"
    
        except psycopg2.Error as error:
    
            print(f"Database connection error! \n-- {error}\nPlease try again!") 

        except KeyboardInterrupt:

            print("Interrupted")   



    def iss_data_streaming(self):
        
        self.psql_query = """INSERT INTO iss_monitoring ("iss_lat", "iss_long", "current_country", "iss_time", "current_time", "current_date") VALUES (%s, %s, %s, %s, %s, %s);"""
        self.warning = "Connection database OK!\nStarting the streaming... \nPress CTRL+C to stop."
        self.data = None
        self.data_err = None
        self.cursor = None


        try:

            self.iss_database_connection() if self.connection == None else print(self.warning)

            print(self.warning)

            self.cursor = self.connection.cursor()
            
            while True:


                if Iss().response['message'] == 'success':
            
                    self.data = (Iss().position()[0],
                        Iss().position()[1],
                        Astrolabe().iss_country(),
                        Iss().iss_time,
                        datetime.now().strftime('%H:%M:%S'),
                        datetime.now().strftime('%Y-%m-%d')
                    )
            
                    self.cursor.execute(self.psql_query, self.data)
            
                    self.connection.commit()
            
                    time.sleep(30)

                else:

                    self.data_err = (
                        None,
                        None,
                        "COMMUNICATION ERROR",
                        None,
                        datetime.now().strftime('%H:%M:%S'),
                        datetime.now().strftime('%Y-%m-%d')
                    )
            
                    self.cursor.execute(self.psql_query, self.data_err)

                    self.connection.commit()

                    print("ISS Communication Error")

                    time.sleep(30)


        except KeyboardInterrupt:

            if self.connection:
                self.cursor.close()
                self.connection.close()
                print ("PostgreSQL connection is closed")
            pass

    